<a href="https://colab.research.google.com/github/GalaxyHunters/VIVID3D/blob/docs%2Fexamples/docs/source/notebooks/BrainCTScan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Brain CT Scan (NRRD)
Datasets from https://klacansky.com/open-scivis-datasets/

Datasets are generously hosted by the Scientific Computing and Imaging Institute https://sci.utah.edu/

## *Prerequisites* (Run Once)

In [ ]:
#@title Installing Libraries
%%capture
!pip install pynrrd
!pip install vivid3d

In [ ]:
#@title Download Data
from urllib.request import urlretrieve

## Embedded version
data_path = 'http://snoid.sv.vt.edu/medical/X3DVolumes/examples/datasets/nrrd/brain2.nrrd'
data_file = data_path.split('/')[-1]
data_response = urlretrieve(data_path, data_file)

## Model Creation

In [ ]:
#@title Brain (Embedded) and 2D Masking with Matplotlib
import nrrd
import numpy as np
import vivid3d

# Brain:
header_file = 'brain2.nrrd'
data, header = nrrd.read(header_file)

volume = np.flip(data.T)

# Downsampling Volume for faster run
volume_s = np.clip(volume[::3,40:400:3,10:460:3], 0, 200)

min_f = volume_s.min()
max_f = volume_s.max()
print(f'original dimensions: {volume.shape}\noriginal particles: {volume.size}')
print(f'Downsampled dimensions: {volume_s.shape}\nDownsampled particles: {volume_s.size}')
print(f'field range:\nmin: {min_f}, max: {max_f}')

viewer = vivid3d.VolumetricViewer(volume_s)

original dimensions: (230, 512, 512)
original particles: 60293120
Downsampled dimensions: (77, 120, 150)
Downsampled particles: 1386000
field range:
min: 0, max: 200


HTML(value='<link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesom…

AppLayout(children=(VBox(children=(HTML(value='<center><h1>Volumetric Viewer</h1></center>'), HBox(children=(L…

In [ ]:
#@title Vivid3D Model
import vivid3d
import matplotlib.cm as cmaps

# Making Data
w,h,d = volume_s.shape
x,y,z = np.mgrid[:w,:h,:d]
points = np.column_stack((x.ravel(),y.ravel(),z.ravel()))
field = volume_s.ravel()

# Running Voronoi on Data
brain_volume = vivid3d.VoronoiVolume(points, field)

# Extracting IsoSurfaces and adding to model
brain_model = vivid3d.Model()

surfaces = [
    # (min_threshold, max_threshold, alpha)
    (10, 130, 0.3), # Soft Tissue
    (80, max_f, 0.3), # Dense Tissue
    (150, max_f, 0.3), # Densest tissue
]

for surface in surfaces:
  mask = [val >= surface[0] and val <= surface[1] for val in field]
  mesh = brain_volume.iso_surface(mask, str(surface[0]), surface[2])
  mesh.scale([2,1,1])
  mesh.colormap = "RdPu_r"
  mesh.smooth()
  brain_model.add_mesh(mesh)

brain_model.show()

27-10-2022 20:38:44 - VIVID:   Preprocessing Data
27-10-2022 20:38:46 - VIVID:   Begin Compute Voronoi with 1386000 points
27-10-2022 20:45:01 - VIVID:   Completed Voronoi Computation
